<a href="https://colab.research.google.com/github/bthodla/sales_demo/blob/dev/sales_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from google.colab import drive
import os, sys
from dateutil import parser
import datetime
import re

from bokeh.io import show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool
from bokeh.plotting import figure, output_file, output_notebook, show

%matplotlib inline

# To suppress numeric values from being returned in exponential format
pd.options.display.float_format = '{:20,.2f}'.format

plt.style.use('seaborn-whitegrid')

# suppress warnings from final output
import warnings
warnings.simplefilter("ignore")


In [0]:
# !pip install bokeh --upgrade

In [2]:
drive.mount('/content/gdrive')
data_folder = '/content/gdrive/My Drive/Colab Notebooks/sales_demo/'


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
customer_df = pd.read_parquet(data_folder + 'customer.parquet', engine='pyarrow')
contact_df = pd.read_parquet(data_folder + 'contact.parquet', engine='pyarrow')
product_df = pd.read_parquet(data_folder + 'product.parquet', engine='pyarrow')
state_df = pd.read_parquet(data_folder + 'state.parquet', engine='pyarrow')
touch_df = pd.read_parquet(data_folder + 'touch.parquet', engine='pyarrow')
txn_df = pd.read_parquet(data_folder + 'txn.parquet', engine='pyarrow')


In [0]:
# Extract the year and month from "txn_df" and "touch_df" to seprate columns
touch_df['txn_year_month'] = touch_df['txn_date'].map(lambda x: x.to_period('M'))
txn_df['txn_year_month'] = txn_df['txn_date'].map(lambda x: x.to_period('M'))


In [12]:
txn_df.sample(10)


,contact_id,txn_id,txn_date,product_id,sales,redemptions,txn_year_month
44427,542022975,422952925,2015-01-26,PCRIX,0.00,"-8,702.40",2015-01
60481,424317213,422974637,2015-08-19,TEDIX,"3,397.35",0.00,2015-08
69650,208309599,422983806,2015-12-21,FDFFX,"2,046.62",0.00,2015-12
68089,403217176,422982245,2015-11-16,PIREX,"2,586.04",0.00,2015-11
6454,436867170,422914952,2015-05-01,FCISX,"8,419.29",0.00,2015-05
69443,243999332,422983599,2015-08-30,AIVSX,"3,300.64",0.00,2015-08
59460,516634093,422973616,2015-05-11,DLTNX,"5,655.00",0.00,2015-05
42616,280418478,422951114,2015-11-24,MDISX,0.00,"-9,707.16",2015-11
13536,328648413,422922034,2015-07-07,VIVIX,"6,048.44",0.00,2015-07
85610,378981070,422999766,2015-10-10,VSIAX,0.00,"-3,684.85",2015-10


In [6]:
customer_df.info()

"""
category - Qualitative Categorical
"""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3876 entries, 0 to 3875
Data columns (total 3 columns):
customer_id      3876 non-null int64
customer_name    3876 non-null object
category         3876 non-null object
dtypes: int64(1), object(2)
memory usage: 90.9+ KB


In [7]:
contact_df.info()

"""
city - Qualitative Nominal
state - Qualitative Nominal
zip - Qualitative Nominal
status - Qualitative Nominal
opportunity - Quantitative Continuous
"""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210002 entries, 0 to 210001
Data columns (total 12 columns):
contact_id     210002 non-null int64
first_name     210002 non-null object
last_name      210002 non-null object
city           210002 non-null object
state_code     210002 non-null object
zip            210002 non-null object
country        210002 non-null object
latitude       210002 non-null float64
longitude      210002 non-null float64
customer_id    210002 non-null int64
status         210002 non-null object
opportunity    210002 non-null int64
dtypes: float64(2), int64(3), object(7)
memory usage: 19.2+ MB


In [8]:
product_df.info()

"""
tna - Quantitative Continuous
ms_rating - Qualitative Ordinal
exp_ratio - Quantitative Continuous
market_cap - Quantitative Discrete
"""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426 entries, 0 to 425
Data columns (total 6 columns):
product_id      426 non-null object
product_name    426 non-null object
tna             426 non-null float64
ms_rating       426 non-null int64
exp_ratio       426 non-null float64
market_cap      426 non-null object
dtypes: float64(2), int64(1), object(3)
memory usage: 20.0+ KB


In [9]:
state_df.info()

"""
state - Qualitative Nominal
region - Qualitative Nominal
"""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 3 columns):
state_code    50 non-null object
state         50 non-null object
region        50 non-null object
dtypes: object(3)
memory usage: 1.2+ KB


In [10]:
touch_df.info()

"""
txn_date - Quantitative Interval
touch_type - Qualitative Nominal
txn_year_month - Quantitative Interval
"""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16974 entries, 0 to 16973
Data columns (total 6 columns):
contact_id        16974 non-null int64
txn_id            16974 non-null int64
txn_date          16974 non-null datetime64[ns]
product_id        16974 non-null object
touch_type        16974 non-null object
txn_year_month    16974 non-null period[M]
dtypes: datetime64[ns](1), int64(2), object(2), period[M](1)
memory usage: 795.7+ KB


In [11]:
txn_df.info()

"""
txn_date - Quantitative Interval
sales - Quantitative Continuous
redemptions - Quantitative Continuous
txn_year_month - Quantitative Interval
"""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90010 entries, 0 to 90009
Data columns (total 7 columns):
contact_id        90010 non-null int64
txn_id            90010 non-null int64
txn_date          90010 non-null datetime64[ns]
product_id        90010 non-null object
sales             90010 non-null float64
redemptions       90010 non-null float64
txn_year_month    90010 non-null period[M]
dtypes: datetime64[ns](1), float64(2), int64(2), object(1), period[M](1)
memory usage: 4.8+ MB


In [0]:
txn_ym_df = txn_df.groupby('txn_year_month')[['sales', 'redemptions']].sum().reset_index()
txn_ym_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 3 columns):
txn_year_month    15 non-null period[M]
sales             15 non-null float64
redemptions       15 non-null float64
dtypes: float64(2), period[M](1)
memory usage: 440.0 bytes


In [0]:
# Sales and Redemptions trends over time
# output to static HTML file
# output_file("lines.html")

output_notebook()

# create a new plot with a title and axis labels
p = figure(plot_height=300, plot_width=800, tools="xpan", toolbar_location=None,
           x_axis_type="datetime", x_axis_location="below",
           background_fill_color="#efefef")

p.vline_stack(['sales', 'redemptions'], x='txn_year_month', source=txn_ym_df, color = ['blue', 'red'])
p.yaxis.axis_label = 'Amount'

"""
select = figure(title="Drag the middle and edges of the selection box to change the range above",
                plot_height=130, plot_width=800, y_range=p.y_range,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None, background_fill_color="#efefef")

select.line('txn_year_month', 'sales', source=txn_ym_df)
select.ygrid.grid_line_color = None

"""

# show(column(p, select))
show(column(p))

Loading BokehJS ...